## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-04-06-32-00 +0000,wsj,Japan Set for First Female Prime Minister Amid...,https://www.wsj.com/world/asia/japan-prime-min...
1,2025-10-04-06-23-16 +0000,bbc,Sanae Takaichi set to become Japan's first fem...,https://www.bbc.com/news/articles/cx2pmy7m72lo...
2,2025-10-04-06-15-54 +0000,wapo,Sanae Takaichi is set to become Japan’s first ...,https://www.washingtonpost.com/world/2025/10/0...
3,2025-10-04-06-14-58 +0000,nyt,Japan Set for First Female Prime Minister,https://www.nytimes.com/2025/10/04/world/asia/...
4,2025-10-04-06-11-18 +0000,bbc,Green Party leader on holding government to ac...,https://www.bbc.com/news/videos/c62l1j70qlgo?a...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2344/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
33,trump,53
34,hamas,17
313,shutdown,15
174,court,15
402,will,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
41,2025-10-04-00-42-44 +0000,nypost,Israel ready for ‘immediate implementation’ of...,https://nypost.com/2025/10/03/world-news/israe...,132
140,2025-10-03-20-03-20 +0000,nypost,Hamas agrees to release all Israeli hostages p...,https://nypost.com/2025/10/03/world-news/hamas...,132
18,2025-10-04-03-10-45 +0000,nypost,"Hamas internal rift over hostages, weapons cou...",https://nypost.com/2025/10/03/world-news/hamas...,120
80,2025-10-03-22-22-09 +0000,nyt,Trump Calls on Israel to Stop Bombing Gaza Aft...,https://www.nytimes.com/2025/10/03/us/politics...,116
138,2025-10-03-20-27-00 +0000,wsj,The Trump administration has frozen $2.1 billi...,https://www.wsj.com/politics/policy/trump-admi...,116


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
41,132,2025-10-04-00-42-44 +0000,nypost,Israel ready for ‘immediate implementation’ of...,https://nypost.com/2025/10/03/world-news/israe...
259,69,2025-10-03-12-00-08 +0000,wapo,One victim in Yom Kippur synagogue attack in E...,https://www.washingtonpost.com/world/2025/10/0...
138,63,2025-10-03-20-27-00 +0000,wsj,The Trump administration has frozen $2.1 billi...,https://www.wsj.com/politics/policy/trump-admi...
130,61,2025-10-03-20-41-31 +0000,nypost,Sean ‘Diddy’ Combs sentenced to only 4 years i...,https://nypost.com/2025/10/03/us-news/sean-did...
223,59,2025-10-03-14-42-11 +0000,latimes,Supreme Court will decide if gun owners may ca...,https://www.latimes.com/politics/story/2025-10...
0,51,2025-10-04-06-32-00 +0000,wsj,Japan Set for First Female Prime Minister Amid...,https://www.wsj.com/world/asia/japan-prime-min...
227,43,2025-10-03-14-23-50 +0000,nypost,Cops probe whether Manchester synagogue terror...,https://nypost.com/2025/10/03/world-news/manch...
268,37,2025-10-03-11-01-00 +0000,startribune,Unemployed Minnesotans face longer job searche...,https://www.startribune.com/long-term-unemploy...
282,36,2025-10-03-10-00-00 +0000,latimes,She went to get her green card and now faces d...,https://www.latimes.com/california/story/2025-...
150,36,2025-10-03-19-37-00 +0000,wsj,Trump ordered another lethal military strike o...,https://www.wsj.com/politics/national-security...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
